In [39]:
import os
import torch
import pandas as pd
import numpy as np

from transformers import pipeline
from zoo_params import zoo_params, create_name
from model import gptKILLER
from pipeline import find_start_and_end

In [40]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
def get_pipeline():
    return pipeline(
        "question-answering",
        model="Den4ikAI/rubert-tiny-squad",
        tokenizer="Den4ikAI/rubert-tiny-squad"
    )

In [4]:
index = 11
pipeline = get_pipeline()
tokenizer = pipeline.tokenizer
model_params = zoo_params[index]["model"]
model_name = create_name(zoo_params[index])
model_params["pipeline"] = pipeline

In [5]:
chkp_folder = "chkp"
checkpoint = torch.load(os.path.join(chkp_folder, model_name, f'{model_name}.pt'))
model = gptKILLER(**model_params)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)

In [6]:
d_test = pd.read_json("nlp_test_task_2023/dataset/test.json")

In [7]:
d_test

,id,text,label
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта
...,...,...,...
313,854936033,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,обеспечение гарантийных обязательств
314,576390745,Часть III Проект договора Договор №______ пост...,обеспечение гарантийных обязательств
315,323745820,УТВЕРЖДАЮ Председатель единой комиссии по осущ...,обеспечение гарантийных обязательств
316,712286194,Версия с 04.07.2022 года У Т В Е Р Ж Д А Ю «Го...,обеспечение гарантийных обязательств


In [12]:
answers = []
for i in range(len(d_test)):
    context = d_test.iloc[i].text
    question = d_test.iloc[i].label
    tokens = tokenizer(
            question,
            context, 
            return_tensors="pt"
    )
    with torch.no_grad():
        res = model(
            tokens.input_ids.to(device), 
            tokens.token_type_ids.to(device), 
            tokens.attention_mask.to(device), 
            torch.Tensor([0]).long().to(device), 
            torch.Tensor([0]).long().to(device)
        )
    
    answer_start_index = res.start_logits.argmax()
    answer_end_index = res.end_logits.argmax()
    
    predict_answer_tokens = tokens.input_ids[0, answer_start_index : answer_end_index + 1]
    
    answer = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)
    start_answer, end_answer = find_start_and_end(context, answer)
    answer = context[start_answer: end_answer + 1]
    answers.append({
        "text" : [answer],
        "answer_start" : [start_answer],
        "answer_end" : [end_answer]
    })
    

In [25]:
d_test["extracted_part"] = answers

In [35]:
with open("predictions.json", "w") as file:
    file.write(d_test.to_json(orient='records', force_ascii=False))

In [36]:
d_check = pd.read_json("predictions.json")

In [38]:
d_check

,id,text,label,extracted_part
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения Контр...
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта,{'text': ['Размер обеспечение исполнения контр...
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта,{'text': ['Размер обеспечение исполнения контр...
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта,{'text': ['Размер обеспечение исполнения контр...
...,...,...,...,...
313,854936033,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
314,576390745,Часть III Проект договора Договор №______ пост...,обеспечение гарантийных обязательств,{'text': ['Размер обеспечения гарантийных обяз...
315,323745820,УТВЕРЖДАЮ Председатель единой комиссии по осущ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
316,712286194,Версия с 04.07.2022 года У Т В Е Р Ж Д А Ю «Го...,обеспечение гарантийных обязательств,{'text': ['Обеспечение гарантийных обязательст...
